# Test sparse matrix
algo 1

In [100]:
import numpy as np
from helpers import Gerschgorin, check_ortho, check_new_eig
from warnings import warn
from scipy import sparse
import scipy

In [101]:
def Pei_function(alpha, n):
    '''
    input:  alpha: a scalar
            n: an integer
    output: the Pei matrix with dimension n x n
    '''
    ones = np.ones((n, n))
    Pei_matrix=alpha*sparse.eye(n)+sparse.csr_matrix(ones, (n,n))
    return Pei_matrix


In [102]:
a = 1e-3
gamma = 3
A=Pei_function(alpha=5, n=4)

print(A.toarray())

[[6. 1. 1. 1.]
 [1. 6. 1. 1.]
 [1. 1. 6. 1.]
 [1. 1. 1. 6.]]


In [103]:
def T_tilda(T, gamma, a):
    """
    Implements the correction mentioned in Remark 3. 
    INPUT:
    - T: T_j
    - gamma: gamma_j
    - a: scalar, a or b
    OUTPUT:
    - T_tilda_j: matrix as defined page 76 of Bai et al., for the Gauss-Radau case.
    """
    print("T.shape:", T.shape)
    e_j = np.zeros((T.shape[0],1))
    e_j[-1,:] = 1
    e_j = sparse.csr_array(e_j)
    print("e_j.shape:", e_j.shape)
    delta = sparse.linalg.spsolve(T-a*sparse.eye(T.shape[0]), gamma*gamma*e_j) # tridiagonal system, could be written without np.linalg.solve maybe
    phi = a + delta[-1]
    temp_left = sparse.vstack([T, gamma*e_j.T])
    var = sparse.vstack([gamma*e_j, phi])
    print("var.shape:", var.shape)
    temp_right = sparse.csr_matrix(var)
    print("temp_right.shape:", temp_right.shape)
    return sparse.hstack([temp_left, temp_right])

In [112]:
# test T_tilda function: a should become an eigenvalue of T_tilda_j.
a = 1e-3
gamma = 3
A=Pei_function(alpha=5, n=4)

T_tilda_j = T_tilda(A, gamma, a)
print("A=\n", A.toarray())
print("A_tilda=\n", T_tilda_j.toarray())


eigs = scipy.linalg.eig(T_tilda_j.toarray())
print("is a a new eigenvalue of T_tilda_j:", check_new_eig(a, np.sort(eigs[0])))

T.shape: (4, 4)
e_j.shape: (4, 1)
var.shape: (5, 1)
temp_right.shape: (5, 1)
A=
 [[6. 1. 1. 1.]
 [1. 6. 1. 1.]
 [1. 1. 6. 1.]
 [1. 1. 1. 6.]]
A_tilda=
 [[6.         1.         1.         1.         0.        ]
 [1.         6.         1.         1.         0.        ]
 [1.         1.         6.         1.         0.        ]
 [1.         1.         1.         6.         3.        ]
 [0.         0.         0.         3.         1.60129783]]
is a a new eigenvalue of T_tilda_j: True


In [99]:
def algorithm_1(A, u, function, maxit=50, epsilon=1e-5):
    # to do: save T_j as sparse
    '''
    Implements algorithm 1 from Bai et al. It computes a lower/upper bound of the quantity u^T f(A) u by using the Gauss-Radau rules
    INPUT:
    - A: a symmetric positive definite matrix of size n times n for some n, with eigenvalues in [a,b]
    - u: vector of size n 
    - f: smooth function in the interval [a,b]
    - maxit: maximum number of iteration
    - epsilon: tolerance between two iterations
    
    OUTPUT:
    - [U,L]: Upper and Lower bound of the quantity u^T f(A) u by using the Gauss-Radau rule.
    '''
    # Remark 1: compute a and b 
    interval = Gerschgorin(A)

    if interval[0]<=0:
        interval[0] = 1e-4
    #print("a=", interval[0])  Armelle: I've put these two as comments
    #print("b=", interval[1])
    if (np.linalg.eigvals(A)<=0).any():
        warn("The matrix A should be positive definite.")
        print("eigenvalues of A:", np.linalg.eigvals(A))
    if not (A==A.T).all():
        warn("A is not symmetric. Please choose A such that A=A.T")
        print("A =",A)

    # set the first variables
    x_j1 = u/np.linalg.norm(u)    
    gamma_j1 = 0.0
    I_j = np.zeros(2)
    I_j1 = np.zeros(2)

    # stopping criteria for I_j^U and I_j^L
    above_thresh_mask = np.array([True, True]) 
    indices = np.arange(2)

    # Save X as a matrix (for re-orthogonalization)
    n=len(u) 
    X=np.zeros((n, maxit+1))
    X[:,0] = x_j1

    # iteration
    for j in range(maxit):
        w = A@X[:,j]
        alpha_j=X[:,j].T@w
        r_j = w - alpha_j*X[:,j]
        if j>0:
            r_j = r_j - gamma_j1*X[:,j-1]
        
        #reorthogonlization to avoid roundoff error encured by Lanczos
        alphas=X.T@r_j
        r_j=r_j-X@alphas
        alpha_j=alpha_j+alphas[-1]
        gamma_j = np.linalg.norm(r_j)

        # build T_j:
        if j==0:
            T_j = np.array([alpha_j])
        else:
            # horizontal array [0, ..., 0, gamma_{j-1}]
            temp_h = np.expand_dims(np.zeros(T_j.shape[0]),1)
            temp_h[-1] = gamma_j1
            # vertical array [0, ..., 0, gamma_{j-1}, alpha_j].T
            temp_v = np.expand_dims(np.zeros(T_j.shape[0] + 1),1)
            temp_v[-1] = alpha_j
            temp_v[-2] = gamma_j1
            # new T_j:
            T_j = np.hstack((np.vstack((T_j, temp_h.T)), temp_v))
        
        # for Gauss Radau, a or b have to be zeros of the polynomial, i.e. must be eigenvalues of T_tilda_{j'}:
        for i in indices[above_thresh_mask]: # for lower and upper bounds
            T_tilda_j = T_tilda(T_j, gamma_j, interval[i])
            
            # compute eigenvalues of T_tilda_j:            
            theta_k, eig_vec = np.linalg.eigh(T_tilda_j)
            w_k_square = eig_vec[0, :]*eig_vec[0,:]
            I_j[i] = function(theta_k).dot(w_k_square)

            # stopping criterion
            if (j>0) & (np.abs(I_j[i] - I_j1[i]) <= epsilon*np.abs(I_j[i])):
                above_thresh_mask[i] = False
                
        # check stopping criterion  
        if not above_thresh_mask.any(): #when both are false: break.
            break
        
        x_j1 = r_j/gamma_j
        X[:,j+1]=x_j1.copy()
        I_j1 = I_j.copy()
        gamma_j1 = gamma_j.copy()
        
        if not check_ortho(X[:,0:j+1]):
            warn("The algorithm does not build an orthonormal basis, at j ="+str(j))

    return u.dot(u)*I_j   